In [99]:
import numpy as np
import pandas as pd

# Training

### Prompt Preparation

In [8]:
df = pd.read_csv("data/splits/val.csv")
df.head()

,Unnamed: 0,id,tweet_id,aggression,offense,codemixed,tweet_text
0,7624,192426,1.585850e+18,2,1,0,@user Congress is not a political party.. It i...
1,5663,159896,1.581150e+18,2,1,1,@user लगता है सरकार कोई है ही नही...इसपे UAPA ...
2,4575,120505,1.577940e+18,1,1,1,Two Sadhus beaten in Chhatishgarh suspecting ...
3,3970,80001,1.583730e+18,0,0,0,@user @user No they are not. By Muslim they me...
4,2792,50075,1.575170e+18,1,1,0,@user Look is stupid


Exp Prompt:

Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request. ### Instruction: Evaluate this sentence for spelling and grammar mistakes ### Input: He finnished his meal and left the resturant ### Response: There are two spelling errors in the sentence. The corrected sentence should be: "He finished his meal and left the restaurant."

My Prompt:

You are an expert in hate speech detection. Offensive tweets are defined as tweets containing profane words, sarcastic remarks, insults, slanders or slurs. These can have a potentially harmful effect on a given target. Classify the following input tweet as Offensive or Non-Offensive.

`###` Input: <tweet>

`###` Response: Offensive

In [9]:
system_prompt = "You are an expert in hate speech detection. Offensive tweets are defined as tweets containing profane words, sarcastic remarks, insults, slanders or slurs. These can have a potentially harmful effect on a given target. Classify the following input tweet as Offensive or Non-Offensive."
label_map = {1: "Offensive", 0: "Non-Offensive"}

In [10]:
def prepare_prompt(row):
    # Data Format -- https://huggingface.co/datasets/vicgalle/alpaca-gpt4?row=0
    prompt = system_prompt + "\n\n### Input: " + row["tweet_text"] + "\n\n### Response: " + label_map[row["offense"]]
    return prompt

In [5]:
df["text"] = df.apply(lambda row: prepare_prompt(row), axis=1)
df["text"].values[:3]

array(['You are an expert in hate speech detection. Offensive tweets are defined as tweets containing profane words, sarcastic remarks, insults, slanders or slurs. These can have a potentially harmful effect on a given target. Classify the following input tweet as Offensive or Non-Offensive.\n\n### Input: @user Congress is not a political party.. It is a INC Pvt. Ltd. made by royal Gandhi family for loot people and build new scams.. @user @user @user \n\n@user @user\n\n### Response: Offensive',
       'You are an expert in hate speech detection. Offensive tweets are defined as tweets containing profane words, sarcastic remarks, insults, slanders or slurs. These can have a potentially harmful effect on a given target. Classify the following input tweet as Offensive or Non-Offensive.\n\n### Input: @user लगता है सरकार कोई है ही नही...इसपे UAPA लगना चाहिए और साथ ही इसके घर पे बुलडोझर चलना चाहिए..\n\n### Response: Offensive',
       'You are an expert in hate speech detection. Offensive twe

In [6]:
df[["id", "text"]].to_csv("data/llama_test/train.csv", index=False)

### Train

In [7]:
# MODEL: TinyPixel/Llama-2-7B-bf16-sharded --- Model sharded into 14 smaller models ~ 1gb each
#        abhishek/llama-2-7b-hf-small-shards --- 10 shards
# Max Length can go upto 4096

In [8]:
!autotrain llm --help

usage: autotrain <command> [<args>] llm [-h] [--train] [--deploy]
                                        [--inference] [--data_path DATA_PATH]
                                        [--train_split TRAIN_SPLIT]
                                        [--valid_split VALID_SPLIT]
                                        [--text_column TEXT_COLUMN]
                                        [--rejected_text_column REJECTED_TEXT_COLUMN]
                                        [--prompt-text-column PROMPT_TEXT_COLUMN]
                                        [--model MODEL]
                                        [--model-ref MODEL_REF]
                                        [--learning_rate LEARNING_RATE]
                                        [--num_train_epochs NUM_TRAIN_EPOCHS]
                                        [--train_batch_size TRAIN_BATCH_SIZE]
                                        [--warmup_ratio WARMUP_RATIO]
                                        [--gradient_accumulation_s

In [ ]:
!autotrain llm --train --project_name "llama-test" \
              --model TinyPixel/Llama-2-7B-bf16-sharded \
              --data_path data/llama_test \
              --text_column text \
              --use_peft \
              --use_int4 \
              --learning_rate 5e-5 \
              --num_train_epochs 2 \
              --trainer sft \
              --model_max_length 512 \
              --block_size 512 > training.log

# --push_to_hub \
# --repo_id sarx11/llama-test \

In [ ]:
# !autotrain llm \
# --train \
# --model ${MODEL_NAME} \
# --project-name ${PROJECT_NAME} \
# --data-path data/ \
# --text-column text \
# --lr ${LEARNING_RATE} \
# --batch-size ${BATCH_SIZE} \
# --epochs ${NUM_EPOCHS} \
# --block-size ${BLOCK_SIZE} \
# --warmup-ratio ${WARMUP_RATIO} \
# --lora-r ${LORA_R} \
# --lora-alpha ${LORA_ALPHA} \
# --lora-dropout ${LORA_DROPOUT} \
# --weight-decay ${WEIGHT_DECAY} \
# --gradient-accumulation ${GRADIENT_ACCUMULATION} \
# $( [[ "$USE_FP16" == "True" ]] && echo "--fp16" ) \
# $( [[ "$USE_PEFT" == "True" ]] && echo "--use-peft" ) \
# $( [[ "$USE_INT4" == "True" ]] && echo "--use-int4" ) \
# $( [[ "$PUSH_TO_HUB" == "True" ]] && echo "--push-to-hub --token ${HF_TOKEN} --repo-id ${REPO_ID}" )

# Inference

### Load Model

In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM

/home/sn3250/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# checkpoint = "TinyPixel/Llama-2-7B-bf16-sharded"
# tokenizer = AutoTokenizer.from_pretrained(checkpoint)
# model = AutoModelForCausalLM.from_pretrained(checkpoint)

tokenizer = AutoTokenizer.from_pretrained("./llama-test")
tokenizer

LlamaTokenizerFast(name_or_path='./llama-test', vocab_size=32000, model_max_length=512, is_fast=True, padding_side='left', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '</s>'}, clean_up_tokenization_spaces=False),  added_tokens_decoder={
	0: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	1: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
}

In [3]:
model = AutoModelForCausalLM.from_pretrained("./llama-test/") # Loads checkpoint shards

Loading checkpoint shards: 100%|██████████| 14/14 [00:24<00:00,  1.76s/it]


### Load Data for Inference

In [6]:
import numpy as np
import pandas as pd
from collections import Counter

In [7]:
df = pd.read_csv("data/splits/val.csv")
df.head(2)

,Unnamed: 0,id,tweet_id,aggression,offense,codemixed,tweet_text
0,7624,192426,1.585850e+18,2,1,0,@user Congress is not a political party.. It i...
1,5663,159896,1.581150e+18,2,1,1,@user लगता है सरकार कोई है ही नही...इसपे UAPA ...


In [8]:
system_prompt = "You are an expert in hate speech detection. Offensive tweets are defined as tweets containing profane words, sarcastic remarks, insults, slanders or slurs. These can have a potentially harmful effect on a given target. Classify the following input tweet as Offensive or Non-Offensive."
label_map = {1: "Offensive", 0: "Non-Offensive"}

In [9]:
def prepare_prompt_val(row):
    prompt = system_prompt + "\n\n### Input: " + row["tweet_text"] + "\n\n### Response: "
    return prompt

In [10]:
df["text"] = df.apply(lambda row: prepare_prompt_val(row), axis=1)
df["text"].values[:2]

array(['You are an expert in hate speech detection. Offensive tweets are defined as tweets containing profane words, sarcastic remarks, insults, slanders or slurs. These can have a potentially harmful effect on a given target. Classify the following input tweet as Offensive or Non-Offensive.\n\n### Input: @user Congress is not a political party.. It is a INC Pvt. Ltd. made by royal Gandhi family for loot people and build new scams.. @user @user @user \n\n@user @user\n\n### Response: ',
       'You are an expert in hate speech detection. Offensive tweets are defined as tweets containing profane words, sarcastic remarks, insults, slanders or slurs. These can have a potentially harmful effect on a given target. Classify the following input tweet as Offensive or Non-Offensive.\n\n### Input: @user लगता है सरकार कोई है ही नही...इसपे UAPA लगना चाहिए और साथ ही इसके घर पे बुलडोझर चलना चाहिए..\n\n### Response: '],
      dtype=object)

### Test on 1 sample
https://huggingface.co/docs/transformers/main/model_doc/llama#transformers.LlamaForCausalLM

In [11]:
import torch
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader

device = torch.device("cuda") if torch.cuda.is_available() else "cpu"
device

device(type='cuda')

In [12]:
model = model.to(device)
model.eval()

In [33]:
inputs = tokenizer(df["text"][2], padding=True, truncation=True, max_length=512, return_tensors="pt").to(device)

In [34]:
with torch.no_grad():
    generate_ids = model.generate(inputs.input_ids, max_length=300)

In [35]:
tokenizer.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]

'You are an expert in hate speech detection. Offensive tweets are defined as tweets containing profane words, sarcastic remarks, insults, slanders or slurs. These can have a potentially harmful effect on a given target. Classify the following input tweet as Offensive or Non-Offensive.\n\n### Input: Two Sadhus beaten in Chhatishgarh  suspecting they are childlifters. Its a sad thing that Hindu seers are beaten up this way under false charges the moment they try to stop conversion.\nSwami Laxmananand Saraswati was killed in Odisha because he got Ghar wapasi done in thousands\n\n### Response:  Non-Offensive'

### Val Testing

#### Batched

In [26]:
class HSDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        inputs = {key: val[idx] for key, val in self.encodings.items()}
        labels = torch.tensor(self.labels[idx])
        return inputs, labels

    def __len__(self):
        return len(self.labels)

In [27]:
inputs = tokenizer(df['text'].tolist(), padding=True, truncation=True, max_length=512, return_tensors="pt")
inputs = inputs.to(device)

In [28]:
val_dataset = HSDataset(inputs, df['offense'].tolist())
val_dataloader = DataLoader(val_dataset, batch_size=4)

In [30]:
model.eval()
responses = []


for inputs, labels in tqdm(val_dataloader):
    with torch.no_grad():
        generate_ids = model.generate(inputs["input_ids"], max_length=300)
        response = tokenizer.batch_decode(generate_ids, skip_special_tokens=True, 
                                          clean_up_tokenization_spaces=False)[0]
        responses.append(response.split("### Response: ")[1].strip())

100%|██████████| 213/213 [05:21<00:00,  1.51s/it]


In [31]:
responses

['',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 'Non',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '

#### 1 at a time

In [40]:
model.eval()
responses = []

for i in tqdm(range(len(df))):
    inputs = tokenizer(df["text"][i], padding=True, truncation=True, max_length=512, 
                       return_tensors="pt").to(device)
    with torch.no_grad():
        generate_ids = model.generate(inputs.input_ids, max_length=512)
        
    response = tokenizer.batch_decode(generate_ids, skip_special_tokens=True, 
                                      clean_up_tokenization_spaces=False)[0]
    responses.append(response.split("### Response: ")[1].strip())

100%|██████████| 852/852 [05:04<00:00,  2.80it/s]


In [41]:
Counter(responses)

Counter({'Non-Offensive': 846,
         'Offensive': 5,
         '������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������': 1})

In [42]:
df["offense"].value_counts()

offense
0    596
1    256
Name: count, dtype: int64